# Using SQL templating with pydbtools

pydbtools can read SQL files and also rendor SQL with jinja templating this notebook demos both.
> Note this functionality is new to pydbtools v4.0.0.

## Setup

Just run this script to create the source database so we can use it for our example.

In [ ]:
import os
import pandas as pd
import awswrangler as wr
import pydbtools as pydb

In [ ]:
# setup your own testing area (set foldername = GH username)
foldername = "mratford" # GH username
foldername = foldername.lower().replace("-","_")

In [ ]:
bucketname = "alpha-everyone"
s3_base_path = f"s3://{bucketname}/{foldername}/"

db_name = f"aws_example_{foldername}"
source_db_base_path = f"s3://{bucketname}/{foldername}/source_db/"

# Delete all the s3 files in a given path
if wr.s3.list_objects(s3_base_path):
    print("deleting objs")
    wr.s3.delete_objects(s3_base_path)

# Delete the database if it exists
df_dbs = wr.catalog.databases(None)
if db_name in df_dbs["Database"].to_list():
    print(f"{db_name} found deleting")
    wr.catalog.delete_database(
        name=db_name
    )

# Setup source database
# Create the database
wr.catalog.create_database(db_name)

# Iterate through the tables in data/ and write them to our db using awswrangler
for table_name in ["department", "employees", "sales"]:
    
    df = pd.read_csv(f"data/{table_name}.csv")
    table_path = os.path.join(source_db_base_path, f"{table_name}/")
    wr.s3.to_parquet(
        df=df,
        path=table_path,
        index=False,
        dataset=True, # True allows the other params below i.e. overwriting to db.table
        database=db_name,
        table=table_name,
        mode="overwrite",
    )

## Task

We now have a database with 3 tables. We are joing to write an SQL with [Jinja templating](https://realpython.com/primer-on-jinja-templating/) which we can "render" with parameters and then run that query using pydbtools.

### Render a template

Jinja uses the syntax of two curly brackets for it's parameters.

In [ ]:
sql_template = """
SELECT *
FROM {{ db_name }}.{{ table }}
"""
print(pydb.render_sql_template(sql_template, {"db_name": db_name, "table": "department"}))

In [ ]:
# We can now use this rendered SQL to actually return the query.
sql = pydb.render_sql_template(sql_template, {"db_name": db_name, "table": "department"})
pydb.read_sql_query(sql)

In [ ]:
# We can also use the same template to read a different table
sql = pydb.render_sql_template(sql_template, {"db_name": db_name, "table": "sales"})
pydb.read_sql_query(sql)

### Read and render a file

So you might be thinking this seems pointless as we can use f-strings. Which is true. But what does come into use is the ability to read in an SQL file and render it.


In [ ]:
# Let's first create a an sql file
# as your db_name is dependant on who is running this tutorial we'll use an fstring to create a basic SQL file
with open("tempfile.sql", "w") as f:
    f.write(f"SELECT * FROM {db_name}.employees")

Now if you open up the file `tempfile.sql` you'll see a simple SQL file and jupyterlab should also give you some syntax colouring for SQL (because it recognises the `.sql` extension).

With pydbtools you can read an SQL file and then use that query.

In [ ]:
sql = pydb.get_sql_from_file("tempfile.sql")
pydb.read_sql_query(sql)

We can use read in and render SQL templates. This means you can store your SQL template as a file and then just parameterise it when reading it in. Then run it.

First lets overwrite our new file with the SQL template we originally created.

In [ ]:
# Note no f-strings this time. We are using jinja templating
with open("tempfile.sql", "w") as f:
    f.write("SELECT * FROM {{ db_name }}.{{ table_name }}")

Again it is worth looking at the file again (note if it looks the same, close the file in jupyter and reopen it). You should see the same SQL but with the Jinja templating.

Now lets read in and rendor our template.

In [ ]:
sql = pydb.get_sql_from_file("tempfile.sql", jinja_args={"db_name": db_name, "table_name": "department"})
pydb.read_sql_query(sql)

## Conclusion

Writing your SQL as a file can really helpful to utilise syntax highlighting that you don't get when just writing a string. With jinja templating you can create SQL templates and then parameterise them as you please. 

Finally, when you start working with a heavy codebase of SQL across multiple users you can use tools like [sqlfluff](https://github.com/sqlfluff/sqlfluff) to lint your SQL files to ensure your team are working to the same standard of SQL. (_SQL fluff also supports Jinja templated SQL files_)